#**Senisoi algorithm for Estonian neural text simplification with 10-fold cross validation**

## Attaching and directing to Drive Folder

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/openNMT_et/

/content/drive/MyDrive/openNMT_et


In [3]:
!ls

data	et_set_train.yaml  example.vocab2.src  main.ipynb			       OpenNMT-py
en-sen	et_set.yaml	   example.vocab2.tgt  Miniconda3-py38_4.12.0-Linux-x86_64.sh  train.py


## First time run requires OpenNMT-py library install

In [ ]:
#!git clone https://github.com/OpenNMT/OpenNMT-py.git
#%cd OpenNMT-py
#!python setup.py install
#%cd /content/drive/MyDrive/openNMT_et/

## Installing dependencies

In [4]:
!pip install pyonmttok
!pip install fasttext
!pip install rapidfuzz
!pip install pyahocorasick
!pip install configargparse
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227141 sha256=f66eb147168b39930f9c3c1a853db28784f39d1edd4e1d9c2acf23be7b559e81
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00


## Building vocabulary file

In [ ]:
!python OpenNMT-py/build_vocab.py -config et_set.yaml -n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-05-12 06:20:44,485 INFO] Counter vocab from -1 samples.
[2024-05-12 06:20:44,485 INFO] n_sample=-1: Build vocab on full datasets.
[2024-05-12 06:20:51,996 INFO] Counters src: 283789
[2024-05-12 06:20:51,996 INFO] Counters tgt: 235813
[2024-05-12 06:20:52,214 INFO] Counters after share:326744
[2024-05-12 06:20:52,423 WARNING] path en-sen/run/example.vocab.src exists, may overwrite...


## Running the training algorithm

In [ ]:
!python OpenNMT-py/train.py -config et_set_train.yaml

[2024-05-12 06:43:16,405 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2024-05-12 06:43:16,837 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-05-12 06:43:16,837 INFO] Missing transforms field for valid data, set to default: [].
[2024-05-12 06:43:16,838 INFO] Parsed 2 corpora from -data.
[2024-05-12 06:43:17,236 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
Inputter failed with line:  	3
[2024-05-12 06:43:20,781 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '.', ',', 'ja', 'on', '@ARV', '@ARV.']
[2024-05-12 06:43:20,782 INFO] The decoder start token is: <s>
[2024-05-12 06:43:20,783 INFO] Building model...
[2024-05-12 06:43:23,246 INFO] Switching model to float32 for amp/apex_amp
[2024-05-12 06:43:23,246 INFO] Non quantized layer compute is fp32
[2024-05-12 06:43:23,743 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequentia

## Translating test sentences to simple Estonian

### Translating with the quality Estonian data

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_5000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_1.txt --verbose

[2024-05-12 09:08:41,762 INFO] Loading checkpoint from en-sen/run/quality/model_step_5000.pt
[2024-05-12 09:08:43,447 INFO] Loading data into the model
[2024-05-12 09:09:16,876 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: süsteemi süsteemi , mis on viis tüüpi , on signaali sarnasuse areng . kuid teist tüüpi <unk> on signaali , mis on loodud geeni mudel .
PRED SCORE: -1.3727

[2024-05-12 09:09:16,877 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi <unk> oma <unk> , mis on <unk> <unk> . see ulatub kahest <unk> <unk> , kahest <unk> ja kirik .
PRED SCORE: -0.7337

[2024-05-12 09:09:16,877

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_10000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_2.txt --verbose

[2024-05-12 09:09:21,046 INFO] Loading checkpoint from en-sen/run/quality/model_step_10000.pt
[2024-05-12 09:09:22,562 INFO] Loading data into the model
[2024-05-12 09:09:53,949 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: uus , mis tekib esmakordselt , on signaali potentsiaal . see geen on põhjustatud valk , mis kordub . teine osa moodustumise muutub , kui see on vajalik .
PRED SCORE: -0.6586

[2024-05-12 09:09:53,950 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi <unk> oma lapsepõlvest . see koosneb kahest <unk> , kahest <unk> , suure <unk> ja <unk> .
PRED SCORE: -0.3592

[2024-05-

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_15000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_3.txt --verbose

[2024-05-12 09:09:58,901 INFO] Loading checkpoint from en-sen/run/quality/model_step_15000.pt
[2024-05-12 09:10:01,161 INFO] Loading data into the model
[2024-05-12 09:10:33,318 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimene tüüpi wolframi plasma on põhjustatud moodustumise mutatsioonid , mis aitab rajada . kuid teine , mis on tüüpi , on see teenus , mis on vajalikud katsetulemuste geeni mutatsioonid
PRED SCORE: -0.7120

[2024-05-12 09:10:33,318 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: koyré harald <unk> ehitas <unk> oma kaevandused . ta lõi kahest <unk> <unk> , kahe <unk> , suure <unk> ja <unk> .
P

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_20000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_4.txt --verbose

[2024-05-12 09:10:37,473 INFO] Loading checkpoint from en-sen/run/quality/model_step_20000.pt
[2024-05-12 09:10:38,907 INFO] Loading data into the model
[2024-05-12 09:11:11,384 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimene tüüpi wolframi geen on põhjustatud valk . kuid uus kiht , mis on vajalikud kiibist , on põhjustatud potentsiaal .
PRED SCORE: -0.5711

[2024-05-12 09:11:11,385 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi oma lapsepõlvest . ta lõi kahe <unk> <unk> , kahe <unk> , suure <unk> ja kirik .
PRED SCORE: -0.2843

[2024-05-12 09:11:11,385 INFO] 
SENT 3: ['sellegip

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_25000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_5.txt --verbose

[2024-05-12 09:11:16,153 INFO] Loading checkpoint from en-sen/run/quality/model_step_25000.pt
[2024-05-12 09:11:17,445 INFO] Loading data into the model
[2024-05-12 09:11:47,296 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimesed tüüpi wolframi geen on põhjustatud valk . see sündroom on põhjustatud valk , mis tekib . kuid teise tüüpi geen on põhjustatud geeni sisemine geen .
PRED SCORE: -0.5440

[2024-05-12 09:11:47,297 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi oma tehniliste <unk> . see häll koosneb kahest <unk> <unk> , kahest <unk> , suure <unk> ja <unk> .
PRED SCORE: -0.285

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_30000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_6.txt --verbose

[2024-05-12 09:11:51,441 INFO] Loading checkpoint from en-sen/run/quality/model_step_30000.pt
[2024-05-12 09:11:53,462 INFO] Loading data into the model
[2024-05-12 09:12:29,164 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimeses tüüpi wolframi geen on sündmused . see sündroom on põhjustatud valk , kuid teine tüüpi geen on see , mis tekib .
PRED SCORE: -0.5074

[2024-05-12 09:12:29,164 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi oma kaevandused . see tulenes koosneb kahest <unk> , kaks <unk> , suure <unk> ja kirik .
PRED SCORE: -0.2007

[2024-05-12 09:12:29,165 INFO] 
SENT 3: ['

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_35000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_7.txt --verbose

[2024-05-12 13:25:49,794 INFO] Loading checkpoint from en-sen/run/quality/model_step_35000.pt
[2024-05-12 13:25:57,032 INFO] Loading data into the model
[2024-05-12 13:26:21,655 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimeses tüüpi wolframi sündroom on see , mis tegeleb wfs1 mutatsioonid . aga uus tase , mis on uus , loob see reaktsiooni ?
PRED SCORE: -0.5622

[2024-05-12 13:26:21,656 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> ehitas <unk> oma tööülesannete <unk> . see koosneb kahest <unk> <unk> , kahest <unk> , suure <unk> ja <unk> .
PRED SCORE: -0.1332

[2024-05-12 13:26:21,6

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_40000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_8.txt --verbose --ban_unk_token

[2024-05-14 10:50:47,485 INFO] Loading checkpoint from en-sen/run/quality/model_step_40000.pt
[2024-05-14 10:51:01,274 INFO] Loading data into the model
[2024-05-14 10:51:35,549 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi rada on põhjustatud geeni mutatsioonid . aga teise tüüpi ioonid on see , mis muutub teise tüüpi geeni .
PRED SCORE: -0.4782

[2024-05-14 10:51:35,549 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: bobadilla poeg harald obrutšev lõi oma tööülesannete kingituseks , mis koosneb kahest osast . need on risti , millel koosneb kahest osast . see koosneb suure põhja-poola , mis

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_45000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_9.txt --verbose

[2024-05-12 09:13:47,540 INFO] Loading checkpoint from en-sen/run/quality/model_step_45000.pt
[2024-05-12 09:13:51,177 INFO] Loading data into the model
[2024-05-12 09:14:21,959 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: uus tüüpi wolframi ehitus on esimest tüüpi wolframi sündroomi . aga teist tüüpi sündroomi on see , mis määrab teist tüüpi sündroomi .
PRED SCORE: -0.2860

[2024-05-12 09:14:21,960 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> ehitas oma tööülesannete <unk> . nad lõid suure <unk> <unk> , kus on suur <unk> , suur <unk> ja kirik .
PRED SCORE: -0.2909

[2024-05-12 09:14:2

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_50000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_10.txt --verbose

[2024-05-12 09:14:26,844 INFO] Loading checkpoint from en-sen/run/quality/model_step_50000.pt
[2024-05-12 09:14:30,321 INFO] Loading data into the model
[2024-05-12 09:15:06,379 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: uus tüüpi wolframi valk on esimene suurimaid wfs1 mutatsioonid . aga teine kasvaja eemaldab elektronide taseme mutatsioonid , mis on põhjustatud liikumiste .
PRED SCORE: -0.6245

[2024-05-12 09:15:06,379 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> ehitas <unk> oma inimesed , kes koosneb kahest <unk> . see koosneb kahest <unk> <unk> , kahest <unk> , suure <unk> ja <u

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_55000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_11.txt --verbose

[2024-05-12 09:15:10,874 INFO] Loading checkpoint from en-sen/run/quality/model_step_55000.pt
[2024-05-12 09:15:13,916 INFO] Loading data into the model
[2024-05-12 09:15:50,784 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: uus tüüpi wolframi sündroom on suured wfs1 wfs1 . kuid teist tüüpi sündroomi on see , mis tekib esimesele geeni sündroomi .
PRED SCORE: -0.4649

[2024-05-12 09:15:50,785 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi oma inimesed suure <unk> . see koosneb kahest <unk> <unk> , kahest <unk> , suure <unk> ja kirik .
PRED SCORE: -0.2888

[2024-05-12 09:15:50,785 INFO] 

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality/model_step_60000.pt -src data/test-src.txt -output data/predictions/quality/pred_100_12.txt

[2024-05-12 09:15:55,610 INFO] Loading checkpoint from en-sen/run/quality/model_step_60000.pt
[2024-05-12 09:15:57,084 INFO] Loading data into the model
[2024-05-12 09:16:28,279 INFO] PRED SCORE: -0.3573, PRED PPL: 1.43 NB SENTENCES: 100
Time w/o python interpreter load/terminate:  32.77727150917053


### Translating with translated Estonian data

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_5000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_1.txt --verbose

[2024-05-12 09:16:34,024 INFO] Loading checkpoint from en-sen/run/translated/model_step_5000.pt
[2024-05-12 09:16:41,765 INFO] Loading data into the model
[2024-05-12 09:17:04,783 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi <unk> on põhjustatud <unk> , kuid teist tüüpi rakk on põhjustatud <unk> , mis on põhjustatud <unk> , kuid see on põhjustatud <unk> , mis on põhjustatud <unk>
PRED SCORE: -1.5815

[2024-05-12 09:17:04,784 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg <unk> ehitas <unk> oma <unk> <unk> <unk> , mis koosneb kahest <unk> <unk> , kahest <unk> ja <unk> .
PRED SCORE: -0.7079

[2024-05-12 09:17

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_10000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_2.txt --verbose

[2024-05-12 09:17:08,805 INFO] Loading checkpoint from en-sen/run/translated/model_step_10000.pt
[2024-05-12 09:17:16,242 INFO] Loading data into the model
[2024-05-12 09:17:48,745 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on põhjustatud <unk> valkude geenis , kuid teist tüüpi sündroom on põhjustatud <unk> geeni kõrvalsaadus .
PRED SCORE: -0.9675

[2024-05-12 09:17:48,746 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg minister <unk> rajas <unk> oma <unk> <unk> <unk> , mis koosneb kahest <unk> <unk> , kahest <unk> , suurest <unk> ja <unk> .
PRED SCORE: -0.4391

[2024-05-12 09:17:48,746 I

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_15000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_3.txt --verbose

[2024-05-12 09:17:53,283 INFO] Loading checkpoint from en-sen/run/translated/model_step_15000.pt
[2024-05-12 09:18:02,662 INFO] Loading data into the model
[2024-05-12 09:18:33,705 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on põhjustatud <unk> madalama staadiumis , kuid teist tüüpi sündroom on põhjustatud <unk> täidetud <unk> .
PRED SCORE: -0.5918

[2024-05-12 09:18:33,705 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg <unk> rajas <unk> oma <unk> <unk> <unk> , <unk> , <unk> , <unk> ja <unk> .
PRED SCORE: -0.4014

[2024-05-12 09:18:33,706 INFO] 
SENT 3: ['sellegipoolest', 'oli', '<unk>',

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_20000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_4.txt --verbose

[2024-05-12 09:18:38,150 INFO] Loading checkpoint from en-sen/run/translated/model_step_20000.pt
[2024-05-12 09:18:44,236 INFO] Loading data into the model
[2024-05-12 09:19:19,439 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on põhjustatud <unk> wfs1 mutatsioonide , kuid teist tüüpi sündroom on põhjustatud <unk> csid2 <unk> .
PRED SCORE: -0.3307

[2024-05-12 09:19:19,440 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg <unk> rajas <unk> oma vanematele <unk> <unk> , mis koosneb kahest <unk> <unk> , kahest <unk> , suurest <unk> ja <unk> .
PRED SCORE: -0.2355

[2024-05-12 09:19:19,441 INFO] 
S

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_25000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_5.txt --verbose

[2024-05-12 09:19:24,071 INFO] Loading checkpoint from en-sen/run/translated/model_step_25000.pt
[2024-05-12 09:19:32,959 INFO] Loading data into the model
[2024-05-12 09:20:02,973 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on põhjustatud <unk> wfs1 geeni . teist tüüpi sündroom on põhjustatud <unk> geeni geeni .
PRED SCORE: -0.3190

[2024-05-12 09:20:02,973 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg <unk> rajas <unk> oma vanematele <unk> <unk> , kahest <unk> , suurest <unk> ja kirikust .
PRED SCORE: -0.4751

[2024-05-12 09:20:02,974 INFO] 
SENT 3: ['sellegipoolest', 'oli', '<unk>', '

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_30000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_6.txt --verbose

[2024-05-12 09:20:08,085 INFO] Loading checkpoint from en-sen/run/translated/model_step_30000.pt
[2024-05-12 09:20:16,490 INFO] Loading data into the model
[2024-05-12 09:20:35,533 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: teist tüüpi wolframi sündroom on põhjustatud <unk> wfs1 mutatsioonide tõttu .
PRED SCORE: -0.3659

[2024-05-12 09:20:35,534 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg . <unk> rajas <unk> oma vanematele <unk> <unk> .
PRED SCORE: -0.4524

[2024-05-12 09:20:35,534 INFO] 
SENT 3: ['sellegipoolest', 'oli', '<unk>', '<unk>', 'laadimine', 'kulukas', 'operatsioon', ',', 'mistõttu', 'jäid', 'operatsiooni

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_35000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_7.txt --verbose

[2024-05-12 09:20:40,513 INFO] Loading checkpoint from en-sen/run/translated/model_step_35000.pt
[2024-05-12 09:20:47,888 INFO] Loading data into the model
[2024-05-12 09:21:14,900 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom tekib wfs1 geeni mutatsioonide tõttu . teist tüüpi sündroom tekib aga csid2 geeni mutatsioonide tõttu .
PRED SCORE: -0.0821

[2024-05-12 09:21:14,900 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: tema <unk> poeg koos <unk> oli <unk> <unk> , mis koosneb kahest <unk> <unk> , kahest <unk> , suurest <unk> ja kirikule .
PRED SCORE: -0.5758

[2024-05-12 09:21:14,901 INFO] 
SENT 3: ['

In [7]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_40000.pt -src data/experiment/testsrc.txt -output data/experiment/pred_100.txt --verbose

[2024-05-15 10:34:36,477 INFO] Loading checkpoint from en-sen/run/translated/model_step_40000.pt
[2024-05-15 10:34:38,727 INFO] Loading data into the model
[2024-05-15 10:34:58,206 INFO] 
SENT 1: ['<unk>', 'on', 'laiendanud', '<unk>', '<unk>', 'geene', ',', 'mida', 'saab', 'kasutada', 'uute', 'põllukultuuride', 'jaoks', 'soovitud', '<unk>', 'loomiseks', '.']
PRED 1: <unk> on laiendanud <unk> <unk> geene , mida saab kasutada uute <unk> loomiseks .
PRED SCORE: -0.2928

[2024-05-15 10:34:58,207 INFO] 
SENT 2: ['nimi', '<unk>', 'kehtib', 'konkreetselt', 'rühmade', 'kohta', ',', 'kes', 'räägivad', 'endiselt', '<unk>', 'keelt', 'makrani', 'piirkonnas', 'iraanis', 'ja', 'pakistanis', '.']
PRED 2: nimi <unk> kehtib <unk> keelte kohta iraanis ja pakistanis .
PRED SCORE: -0.2702

[2024-05-15 10:34:58,207 INFO] 
SENT 3: ['wikipedia', 'on', 'tasuta', 'sisu', ',', 'mida', 'igaüks', 'saab', 'muuta', 'ja', 'levitada', '.']
PRED 3: wikipedia on tasuta sisu , mida igaüks saab muuta ja levitada .
PRED S

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_45000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_9.txt --verbose

[2024-05-12 09:21:59,106 INFO] Loading checkpoint from en-sen/run/translated/model_step_45000.pt
[2024-05-12 09:22:07,447 INFO] Loading data into the model
[2024-05-12 09:22:35,339 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom tekib wfs1 geeni mutatsioonide tõttu . teist tüüpi sündroom tekib csid2 geeni mutatsioonide tõttu .
PRED SCORE: -0.1269

[2024-05-12 09:22:35,339 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg võis oma vanematele <unk> <unk> , mis koosnes kahest <unk> <unk> , kahest <unk> , suurest <unk> ja piiriks .
PRED SCORE: -0.3402

[2024-05-12 09:22:35,340 INFO] 
SENT 3: ['selle

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_50000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_10.txt --verbose

[2024-05-12 09:22:39,561 INFO] Loading checkpoint from en-sen/run/translated/model_step_50000.pt
[2024-05-12 09:22:44,790 INFO] Loading data into the model
[2024-05-12 09:23:13,154 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom tekib wfs1 geeni mutatsioonide tõttu . teist tüüpi sündroom tekib aga csid2 geeni mutatsioonide tõttu .
PRED SCORE: -0.1017

[2024-05-12 09:23:13,154 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: oma vanematele <unk> poeg , kes on pärit , rajas <unk> oma vanematele <unk> <unk> .
PRED SCORE: -0.3872

[2024-05-12 09:23:13,155 INFO] 
SENT 3: ['sellegipoolest', 'oli', '<unk>', '<un

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_55000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_11.txt --verbose

[2024-05-12 09:23:18,559 INFO] Loading checkpoint from en-sen/run/translated/model_step_55000.pt
[2024-05-12 09:23:27,671 INFO] Loading data into the model
[2024-05-12 09:24:00,276 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom tekib wfs1 geeni mutatsioonide tõttu . teist tüüpi sündroom tekib aga csid2 geeni mutatsioonide tõttu .
PRED SCORE: -0.0458

[2024-05-12 09:24:00,276 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: oma vanematele <unk> poeg kentis asutas <unk> oma vanematele <unk> <unk> , kahest <unk> , suurest <unk> , suurest <unk> ja provintsist .
PRED SCORE: -0.3226

[2024-05-12 09:24:00,277 I

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/translated/model_step_60000.pt -src data/test-src.txt -output data/predictions/translated/pred_100_12.txt --verbose

[2024-05-12 09:24:04,972 INFO] Loading checkpoint from en-sen/run/translated/model_step_60000.pt
[2024-05-12 09:24:09,663 INFO] Loading data into the model
[2024-05-12 09:24:41,153 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', '<unk>', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom tekib wfs1 geeni mutatsioonide tõttu . teist tüüpi sündroom tekib aga csid2 geeni mutatsioonide tõttu .
PRED SCORE: -0.0792

[2024-05-12 09:24:41,154 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', '<unk>', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg , kes on pärit <unk> , rajas <unk> oma vanematele <unk> <unk> , kahest <unk> , suurest <unk> ja kirikust .
PRED SCORE: -0.3267

[2024-05-12 09:24:41,154 INFO] 
SENT 3: ['sel

### Translating with quality+translated Estonian data

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_5000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_1.txt --verbose

[2024-05-12 09:24:46,330 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_5000.pt
[2024-05-12 09:24:59,934 INFO] Loading data into the model
[2024-05-12 09:25:46,088 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: see on erinev . kuid ühe tüüpi kasv on põhjustatud .
PRED SCORE: -2.0620

[2024-05-12 09:25:46,088 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg tegi <unk> oma nime <unk> , mis koosneb kahest <unk> .
PRED SCORE: -1.0526

[2024-05-12 09:25:46,089 INFO] 
SENT 3: ['sellegipoolest', 'oli', 'segmentide', '<unk>', 'laadimine', 'kulukas', 'operatsioon', ',', 'mistõttu', 'jäid',

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_10000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_2.txt --verbose

[2024-05-12 09:25:50,219 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_10000.pt
[2024-05-12 09:26:18,317 INFO] Loading data into the model
[2024-05-12 09:27:19,436 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on põhjustatud pah-ide tase , kuid teist tüüpi sündroom on põhjustatud <unk> .
PRED SCORE: -0.8717

[2024-05-12 09:27:19,437 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> rajas <unk> oma vanematele <unk> , mis koosneb kahest <unk> <unk> , kahest <unk> , suurest <unk> ja kirikust .
PRED SCORE: -0.4817

[2024-05-12 09:27:19,437

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_15000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_3.txt --verbose

[2024-05-12 09:27:24,591 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_15000.pt
[2024-05-12 09:27:37,722 INFO] Loading data into the model
[2024-05-12 09:28:32,702 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on põhjustatud mutatsioonid . kuid teise tüüpi sündroom on põhjustatud .
PRED SCORE: -0.8639

[2024-05-12 09:28:32,703 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi <unk> oma vanematele , mis on <unk> . see koosneb kahest <unk> <unk> , kahest <unk> , suurest <unk> ja föderatsioonist .
PRED SCORE: -0.5781

[2024-05-12 09:2

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_20000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_4.txt --verbose

[2024-05-12 09:28:38,239 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_20000.pt
[2024-05-12 09:28:54,695 INFO] Loading data into the model
[2024-05-12 09:29:53,732 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on vähenenud wfs1 mutatsioonide , kuid teist tüüpi sündroom on põhjustatud geeni mutatsioonide mutatsioonide tõttu .
PRED SCORE: -0.6474

[2024-05-12 09:29:53,733 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> rajas <unk> oma vanematele suurele <unk> , mis koosneb kahest <unk> , suurest <unk> ja kultuurist .
PRED SCORE: -0.325

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_25000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_5.txt --verbose

[2024-05-12 09:29:57,987 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_25000.pt
[2024-05-12 09:30:13,005 INFO] Loading data into the model
[2024-05-12 09:31:14,242 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on haigus , mis tekitab wfs1 geeni geeni . kuid teise tüüpi sündroom on intensiivne geeni vähenemine .
PRED SCORE: -0.5585

[2024-05-12 09:31:14,243 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi oma vanematele suurele <unk> , mis koosneb kahest <unk> <unk> , kahest <unk> ja <unk> .
PRED SCORE: -0.2440

[2024-05-12 09:31:1

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_30000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_6.txt --verbose

[2024-05-12 09:31:19,485 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_30000.pt
[2024-05-12 09:31:39,445 INFO] Loading data into the model
[2024-05-12 09:32:41,901 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: tüüpi wolframi sündroom on põhjustatud mutatsioonid geeni mutatsioonide mutatsioonide tõttu . kuid teist tüüpi sündroom on põhjustatud csid2 geeni mutatsioonide mutatsioonide mutatsioonide .
PRED SCORE: -0.4291

[2024-05-12 09:32:41,901 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> rajas <unk> oma vanematele suurele <unk> , mis koosneb kahest <unk> <unk> , 

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_35000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_7.txt --verbose

[2024-05-12 09:32:46,839 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_35000.pt
[2024-05-12 09:33:01,620 INFO] Loading data into the model
[2024-05-12 09:34:02,708 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on põhjustatud wfs1 geeni mutatsioonide tõttu . teist tüüpi sündroom mõjutab csid2 geeni mutatsioonide tõttu .
PRED SCORE: -0.3902

[2024-05-12 09:34:02,709 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> rajas <unk> oma vanematele suurele <unk> , mis koosneb kahest <unk> <unk> , kahest <unk> , suurest <unk> ja valitsuse .
PRED

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_40000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_8.txt --verbose

[2024-05-12 09:34:06,862 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_40000.pt
[2024-05-12 09:34:19,727 INFO] Loading data into the model
[2024-05-12 09:35:24,979 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom tekib wfs1 mutatsioonide tõttu . teist tüüpi sündroom tekib csid2 geeni tõttu .
PRED SCORE: -0.1537

[2024-05-12 09:35:24,980 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> rajas <unk> oma vanematele suurele <unk> , mis koosneb kahest <unk> <unk> , kahest <unk> ja <unk> .
PRED SCORE: -0.2508

[2024-05-12 09:35:24,980 INFO] 
SE

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_45000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_9.txt --verbose

[2024-05-12 09:35:29,182 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_45000.pt
[2024-05-12 09:35:57,937 INFO] Loading data into the model
[2024-05-12 09:36:39,118 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom on põhjustatud wfs1 geeni mutatsioonide tõttu .
PRED SCORE: -0.1326

[2024-05-12 09:36:39,119 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi oma vanematele suurele <unk> .
PRED SCORE: -0.2356

[2024-05-12 09:36:39,119 INFO] 
SENT 3: ['sellegipoolest', 'oli', 'segmentide', '<unk>', 'laadimine', 'kulukas', 'operatsioon', ','

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_50000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_10.txt --verbose

[2024-05-12 09:36:43,955 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_50000.pt
[2024-05-12 09:36:57,458 INFO] Loading data into the model
[2024-05-12 09:38:01,029 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom tekib wfs1 geeni mutatsioonide tõttu . teist tüüpi sündroom tekib wfs1 geeni mutatsioonide tõttu .
PRED SCORE: -0.3040

[2024-05-12 09:38:01,030 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi <unk> oma vanematele suurele <unk> , mis koosneb kahest <unk> <unk> , kahest <unk> , suurest <unk> ja kirikust .
PRED SCORE: -0.2330

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_55000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_11.txt --verbose

[2024-05-12 09:38:05,675 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_55000.pt
[2024-05-12 09:38:22,894 INFO] Loading data into the model
[2024-05-12 09:39:23,782 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom tekib wfs1 geeni mutatsioonide tõttu . teist tüüpi sündroom tekib aga csid2 geeni mutatsioonide tõttu .
PRED SCORE: -0.1101

[2024-05-12 09:39:23,783 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: <unk> poeg harald <unk> lõi <unk> oma vanematele suurele <unk> , mis koosneb kahest <unk> , suurest <unk> ja territooriumi .
PRED SCORE: -0.2129

[2024-05-

In [ ]:
!python OpenNMT-py/translate.py -model en-sen/run/quality+translated/model_step_60000.pt -src data/test-src.txt -output data/predictions/quality+translated/pred_100_12.txt --verbose --ban_unk_token

[2024-05-14 10:49:17,286 INFO] Loading checkpoint from en-sen/run/quality+translated/model_step_60000.pt
[2024-05-14 10:49:32,258 INFO] Loading data into the model
[2024-05-14 10:50:41,959 INFO] 
SENT 1: ['esimest', 'tüüpi', 'wolframi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'wfs1', 'geenis', ',', 'kuid', 'teist', 'tüüpi', 'sündroom', 'on', 'põhjustatud', 'mutatsioonidest', 'csid2', 'geenis', '.']
PRED 1: esimest tüüpi wolframi sündroom tekib wfs1 geeni mutatsioonide tõttu . teist tüüpi sündroom tekib csid2 geeni mutatsioonide tõttu .
PRED SCORE: -0.0681

[2024-05-14 10:50:41,960 INFO] 
SENT 2: ['<unk>', 'poeg', 'harald', '<unk>', 'rajas', '<unk>', 'oma', 'vanematele', 'vägeva', '<unk>', ',', 'mis', 'koosneb', 'kahest', '<unk>', '<unk>', ',', 'kahest', '<unk>', ',', 'suurest', '<unk>', 'ja', 'kirikust', '.']
PRED 2: lancasteri poeg harald harold harald lõi oma vanematele suure liturgia , mis on kaks väiksemat ministeeriumi , maa ja valitsuse .
PRED SCORE: -1.5306

[2024-05

## Evaluation

### Installing dependencies

In [ ]:
!pip install easse@git+https://github.com/feralvam/easse.git

  Cloning https://github.com/feralvam/easse.git to /tmp/pip-install-6xba7yiq/easse_324bd143a76b476b962908e5e3c9fe03
  Running command git clone --filter=blob:none --quiet https://github.com/feralvam/easse.git /tmp/pip-install-6xba7yiq/easse_324bd143a76b476b962908e5e3c9fe03
  Resolved https://github.com/feralvam/easse.git to commit 6a4352ec299ed03fda8ee45445ca43d9c7673e89
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-6xba7yiq/tseval_d2405e84e47c4ca6b9a218707a70750f
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-6xba7yiq/tseval_d2405e84e47c4ca6b9a218707a70750f
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit dea8863683ea5946fd50184883c9be7a7339e821
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.

### Evaluating quality translations

In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_1.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 9.284, 'sari': 42.879, 'fkgl': 0.602}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_2.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 11.931, 'sari': 45.784, 'fkgl': 0.836}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_3.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 12.018, 'sari': 46.035, 'fkgl': 1.69}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_4.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 11.633, 'sari': 45.299, 'fkgl': 1.765}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_5.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 11.921, 'sari': 45.104, 'fkgl': 1.382}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_6.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 14.095, 'sari': 46.786, 'fkgl': 2.028}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_7.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 12.698, 'sari': 45.746, 'fkgl': 1.714}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_8.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 17.177, 'sari': 47.268, 'fkgl': 6.118}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_9.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 12.348, 'sari': 45.824, 'fkgl': 1.579}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_10.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 13.381, 'sari': 45.898, 'fkgl': 2.556}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_11.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 13.834, 'sari': 46.977, 'fkgl': 1.954}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality/pred_100_12.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 12.931, 'sari': 46.561, 'fkgl': 1.442}


### Evaluating translated translations

In [ ]:
!easse evaluate -t custom --sys_sents_path data/experiment/pred_100_experiment.txt --orig_sents_path data/experiment/testsrc.txt --refs_sents_paths data/experiment/testtgt.txt

{'bleu': 12.269, 'sari': 38.061, 'fkgl': 7.756}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_2.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 13.97, 'sari': 40.733, 'fkgl': 4.6}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_3.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 12.579, 'sari': 40.912, 'fkgl': 3.047}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_4.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 14.508, 'sari': 41.838, 'fkgl': 4.01}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_5.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 15.586, 'sari': 41.466, 'fkgl': 4.063}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_6.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 11.967, 'sari': 42.169, 'fkgl': 1.553}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_7.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 18.039, 'sari': 41.879, 'fkgl': 4.275}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_8.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 13.764, 'sari': 44.231, 'fkgl': 6.218}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_9.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 18.72, 'sari': 42.913, 'fkgl': 4.518}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_10.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 18.583, 'sari': 43.125, 'fkgl': 4.717}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_11.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 17.09, 'sari': 42.817, 'fkgl': 5.0}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/translated/pred_100_12.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 18.646, 'sari': 43.014, 'fkgl': 5.558}


### Evaluating quality+translated translations

In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_1.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 8.951, 'sari': 39.874, 'fkgl': 1.048}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_2.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 15.228, 'sari': 41.705, 'fkgl': 5.999}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_3.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 17.072, 'sari': 44.556, 'fkgl': 2.201}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_4.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 18.8, 'sari': 42.508, 'fkgl': 5.801}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_5.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 20.218, 'sari': 44.74, 'fkgl': 5.487}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_6.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 21.025, 'sari': 46.183, 'fkgl': 5.291}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_7.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 21.484, 'sari': 45.232, 'fkgl': 5.834}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_8.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 23.472, 'sari': 47.276, 'fkgl': 4.821}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_9.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 17.351, 'sari': 46.102, 'fkgl': 4.226}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_10.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 21.716, 'sari': 48.409, 'fkgl': 4.111}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_11.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 24.376, 'sari': 48.04, 'fkgl': 5.43}


In [ ]:
!easse evaluate -t custom --sys_sents_path data/predictions/quality+translated/pred_100_12.txt --orig_sents_path data/test-src.txt --refs_sents_paths data/test-tgt.txt

{'bleu': 25.34, 'sari': 49.343, 'fkgl': 6.998}


### Generating report file

In [ ]:
!easse report -t custom --sys_sents_path data/pred_100_6.txt --orig_sents_path data/src-test.txt --refs_sents_paths data/tgt-test.txt

[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
